## Quadratic Logistic Model, d-optimal

In [1]:
using PyPlot
using LinearAlgebra

#objective function h to minimize
function infmat(x)
    n = Int((length(x)+1)/2)
    a = x[1:n]
    b = x[(n+1):(2*n-1)]
    b = [b; 1-sum(b)]
    p = length(theta)
    mat = zeros(p,p)
    for i in 1:n
        c1 = a[i]*theta[3]*exp(-theta[1]*a[i])
        c2 = -a[i]*theta[3]*exp(-theta[2]*a[i]) 
        c3 = exp(-theta[2]*a[i])-exp(-theta[1]*a[i])
        f = [c1, c2, c3]
        mat = mat + b[i] .* f*f'
    end
    return mat
end

# projection function onto the design space
function proj(x, design)
    n = Int((length(x)+1)/2)
    a = x[1:n]
    b = x[(n+1):(2*n-1)]
    b = [b; 1-sum(b)]
    
    a = max.(a,design[1])
    a = min.(a,design[2])
    
    u = sort(b, rev=true)
    j = n
    while true
        if u[j] + 1/j*(1-cumsum(u)[j]) > 0
            break
        end
        j -= 1
    end
    lambda = 1/j*(1-cumsum(u)[j])
    b = max.(b .+ lambda,0)
    b = b[1:(n-1)]
    
    return [a;b]
end

proj (generic function with 1 method)

In [2]:
#Information matrix for quadratic logistic model
function infmat(x)
    n = Int((length(x)+1)/2)
    a = x[1:n]
    b = x[(n+1):(2*n-1)]
    b = [b; 1-sum(b)]
    p = length(theta)
    mat = zeros(p,p)
    for i in 1:n
        c = exp(theta[1]+theta[2]*(a[i]-theta[3])^2)
        f = [1,(a[i]-theta[3])^2,2*theta[2]*(theta[3]-a[i]) ]
        mat = mat + b[i] * c/(1+c)^2 * f*f'
    end
    return mat
end

infmat (generic function with 1 method)

In [3]:
function diter(k, nptc, niter, design)
    #initialize
    #srand(1992)
    tau = 0.9:(-0.5/niter):0.4
    v = zeros(nptc, 2k-1)
    x = (design[2]-design[1])*rand(nptc*k) .+ design[1]
    x = reshape(x,nptc,k)
    w = rand(nptc*(k-1))
    w = reshape(w,nptc,k-1)
    ptc = hcat(x,w)
    for j in 1:nptc
        ptc[j,:] = proj(ptc[j,:],design)
    end
    pbest = ptc
    pbesth = zeros(nptc)
    for i in 1:nptc
        pbesth[i] = det(infmat(pbest[i,:]))
    end
    gbesth = findmax(pbesth)[1]
    gbest = pbest[findmax(pbesth)[2],:]

    #iterate
    for i in 1:niter # iteration number
        for j in 1:nptc # particle number
            v[j,:] = tau[i] .*v[j,:] + 2 .*rand(2k-1) .*(pbest[j,:]-ptc[j,:]) + 2 .*rand(2k-1) .*(gbest-ptc[j,:])
            ptc[j,:] = ptc[j,:] + v[j,:]
            ptc[j,:] = proj(ptc[j,:],design)
            fit = det(infmat(ptc[j,:]))
            if fit > gbesth
                gbest = pbest[j,:] = ptc[j,:]
                gbesth = pbesth[j] = fit
            elseif fit > pbesth[j]
                pbest[j,:] = ptc[j,:]
                pbesth[j] = fit
            end
        end
    end
    supp = gbest[1:k]
    prob = gbest[(k+1):2k-1]
    prob = [prob; 1-sum(prob)]
    return [supp, prob, gbesth]
end

diter (generic function with 1 method)

In [4]:
theta = [2,3,0]

diter(3, 128, 150, [-3,1])

3-element Array{Any,1}:
  [4.6152e-9, 0.726988, -0.726988]
  [0.333333, 0.333333, 0.333333]  
 5.694646494958843e-5             

In [5]:
result = zeros(10)
for nk = 1:10
    t1 = time()
    diter(nk, 128, 150, [-3,1])
    t2 = time()
    result[nk] = t2-t1
end
result

10-element Array{Float64,1}:
 0.29611992835998535
 0.354733943939209  
 0.40833401679992676
 0.47832489013671875
 0.5228722095489502 
 0.5836188793182373 
 0.6494441032409668 
 0.762814998626709  
 0.8386690616607666 
 0.8963398933410645 

In [6]:
theta = [3,-5,0]

result = diter(4, 256, 200, [-1,1])

3-element Array{Any,1}:
  [-0.57935, -1.0, 0.848815, 1.0]    
  [0.333333, 0.333333, 0.333333, 0.0]
 0.018228167200968273                

In [7]:
result = zeros(100)
for nk = 1:100
    t1 = time()
    diter(nk, 256, 200, [-1,1])
    t2 = time()
    result[nk] = t2-t1
end
result

100-element Array{Float64,1}:
  0.7732818126678467
  1.0059409141540527
  1.0950031280517578
  1.2966480255126953
  1.3905830383300781
  1.543259859085083 
  1.721649169921875 
  2.0142459869384766
  2.2030270099639893
  2.3421900272369385
  2.5218710899353027
  2.686210870742798 
  2.882707118988037 
  ⋮                 
 15.85981798171997  
 16.084355115890503 
 15.877058029174805 
 16.238399982452393 
 18.832539081573486 
 18.960060119628906 
 18.970249891281128 
 16.486989974975586 
 17.88095498085022  
 16.578304052352905 
 16.54472589492798  
 17.548291206359863 